## Classification (c)

In [1]:
# from neuralnet_update_PC import *
# from neuralnet_works_FC import *
from neuralnet import *
from random import shuffle
import numpy as np
import matplotlib.pyplot as plt

config_c = {}
config_c['layer_specs'] = [3072, 64, 64, 10]
config_c['activation'] = 'tanh'
config_c['learning_rate'] = 0.005
config_c['batch_size'] = 128
config_c['epochs'] = 100  
config_c['early_stop'] = True 
config_c['early_stop_epoch'] = 5
config_c['L2_penalty'] = 0  
config_c['momentum'] = True  
config_c['momentum_gamma'] = 0.9  
# Create the model

#TODO
#momentum, early stop , expect accuracy around 37%

In [2]:
"""
Train your model here.
Implement batch SGD to train the model.
Implement Early Stopping.
Use config to set parameters for training like learning rate, momentum, etc.
"""
def train_e(model, x_train, y_train, x_valid, y_valid, config, patience=5):
    epochs = config_c['epochs']
    batch_size = config['batch_size']
    momentum =    config['momentum']
    momentum_gamma = config['momentum_gamma']
    L2_penalty = config['momentum_gamma']
    patience = config['early_stop_epoch']

    train_loss_record = []
    train_accuracy_record = []
    holdout_loss_record = []
    holdout_accuracy_record = []

    # How many times the validation loss has gone up in a row.
    cur_loss_up_sequence = 0

    print("num_mini-batches", len(x_train) / batch_size)

    for epoch in range(epochs):
        #model.zero_grad()
        batch_loss = []
        batch_accuracy = []
        for x, y in generate_minibatches(x_train, y_train, batch_size):

            # print("x:", x.shape)
            # print("y:", y.shape)
            # Forward Pass
            train_y, loss = model.forward(x, y)
            batch_loss.append(loss) 
            # Backward Pass
            model.backward()

            batch_accuracy.append(model.accuracy(x,y))

        model.updateweight() # update weight for each layer.

        model.zero_grad()

        y_hat, loss = model.forward(x_train, y_train)
        acc = model.accuracy(x_train, y_train)
        
        # train_loss = np.mean(np.array(batch_loss))
        # train_accuracy = np.mean(np.array(batch_accuracy))

        train_loss = np.mean(np.array(loss))
        train_accuracy = np.mean(np.array(acc))
        
        holdout_loss = model.forward(x_valid, y_valid)[1]
        holdout_accuracy = model.accuracy(x_valid, y_valid)

        train_loss_record.append(train_loss)
        train_accuracy_record.append(train_accuracy)

        holdout_loss_record.append(holdout_loss)
        holdout_accuracy_record.append(holdout_accuracy)

        print(f' epoch: {epoch + 1}, train accuracy: {train_accuracy:.4f}, train_loss_norm:{train_loss:.4f}, '\
            f'valid_acc: {holdout_accuracy:.4f}, valid_loss_norm: {holdout_loss:.4f}')   
        print()

        # Save the best weights according to test set.
        if holdout_loss > max(holdout_loss_record):
            cur_loss_up_sequence += 1

            if cur_loss_up_sequence >= patience:
                model.save_load_weight(save=False)
                print("earlystop")
                break
        else:
            cur_loss_up_sequence = 0
            # Save the best weights.
            model.save_load_weight(save=True)
    
    return train_loss_record, train_accuracy_record, holdout_loss_record, holdout_accuracy_record

model_c  = Neuralnetwork(config_c)

# Load the data
x_train, y_train, stats = load_data(path="./data",stats = None, mode="train")
x_test, y_test = load_data(path="./data",stats = stats, mode="test")

x_train, y_train, x_valid, y_valid = split_data(x_train,y_train)

train_loss_record, train_accuracy_record, holdout_loss_record, holdout_accuracy_record = train_e(model_c,x_train,y_train,x_valid,y_valid,config_c)

# Recall parameters with minimum validation loss
model_c.save_load_weight(save=False)
test_accuracy = test(model_c, x_test, y_test) 
print(test_accuracy)


(50000, 32, 32, 3)
inp: (50000, 32, 32, 3)
num_mini-batches 351.5625
d_w: 2.220446049250313e-14
d_b: -6.661338147750939e-16
pre_d_w: 0.0
pre_d_b: 0.0
d_w: 8.543056850616104
d_b: 8.731248782971136
pre_d_w: 0.0
pre_d_b: 0.0
d_w: -6917.993034387071
d_b: 1.1677161207092877
pre_d_w: 0.0
pre_d_b: 0.0
 epoch: 1, train accuracy: 0.1138, train_loss_norm:10.1753, valid_acc: 0.1060, valid_loss_norm: 10.3014

d_w: 0.0
d_b: 5.575401251789458e-15
pre_d_w: 2.220446049250313e-14
pre_d_b: -6.661338147750939e-16
d_w: 0.0
d_b: 0.0
pre_d_w: 8.543056850616104
pre_d_b: 8.731248782971136
d_w: 0.0
d_b: 0.0
pre_d_w: -6917.993034387071
pre_d_b: 1.1677161207092877
 epoch: 2, train accuracy: 0.1406, train_loss_norm:2.2970, valid_acc: 0.1440, valid_loss_norm: 2.2975

d_w: 6.689093723366568e-15
d_b: -7.355227538141662e-16
pre_d_w: 0.0
pre_d_b: 5.575401251789458e-15
d_w: -7.815672953389912
d_b: 0.10718037041087011
pre_d_w: 0.0
pre_d_b: 0.0
d_w: 93.69447867015944
d_b: 0.014897932811446751
pre_d_w: 0.0
pre_d_b: 0.0
 e

In [ ]:

plt.figure(1)
plt.plot(np.arange(config_c['epochs']), train_loss_record, label='train')
plt.plot(np.arange(config_c['epochs']), holdout_loss_record, label='valid')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and validation loss vs epochs')
plt.legend()
plt.show()


plt.figure(2)
plt.plot(np.arange(config_c['epochs']), train_accuracy_record, label='train')
plt.plot(np.arange(config_c['epochs']), holdout_accuracy_record, label='valid')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and validation accuracy vs epochs')
plt.legend()
plt.savefig("plots/PA2_c_2.png")
plt.show()